In [8]:
using BenchmarkTools;
using LaTeXStrings
using Colors, Plots, Images;
col = palette(:default);
(cgrad::PlotUtils.ContinuousColorGradient)(xs::Vector{Vector{Float64}}) = [cgrad[x] for x in xs];
(cgrad::PlotUtils.ContinuousColorGradient)(m::Matrix{Float64}) = reshape(cgrad[m[:]], size(m));
using CUDA

In [18]:
function logsumexp_cu(x; dim)
    c = maximum(x)
    return c .+ log.(sum(exp.(x .- c), dims=dim))
end

function gaussian_logpdf(x, mu, sig)
    d = (x .- mu).^2 ./ sig.^2
    log_p = - log.(sqrt.(sig * 2 * π)) .- 1/2 * d
    return log_p
end;

In [10]:

x   = CUDA.ones(100, 3)
mu  = CUDA.ones(100, 3)
sig = CUDA.ones(100, 3)

logx = gaussian_logpdf(x, mu, sig)
size(logx)


(100, 3)

In [15]:
x   = CUDA.ones(100, 3, 1, 1)
mu  = CUDA.ones(  1, 1, 7, 1)
sig = CUDA.ones(  1, 1, 1, 5)

logx = gaussian_logpdf(x, mu, sig)
size(logx)



50.0

In [52]:
x = CUDA.rand(1,3)
x = reshape(x, 1, 3, 1)

CUDA.allowscalar(false)

CUDA.@allowscalar x[1]

0.45828736f0

In [38]:
k = 2
n = 7
m = 3

x = CUDA.rand(1, n, 1, 2)
y = CUDA.rand(k, n, m, 2)

sig     = CUDA.rand(1,1,1,1,3,1)
outlier = CUDA.rand(1,1,1,1,1,3)
sig = 1.0
outlier=0.5
outlier_vol = 1.0

# Line by line...
# 1. Compute 1D Gaussians - (n,m,2)
# 2. Convert to 2D gausians - (n,m)
# 3. Convert to mixture of m 2D gausians (GM) - (n,)
# 4. Convert to mixture of `GM` and `anywhere` (D) - (n,)
# 5. Convert to Product of D's - ()
log_p = gaussian_logpdf(x, y, sig)
log_p = sum(log_p, dims=4)
log_p = logsumexp_cu(log_p .- log(m), dim=3)
log_p_or_any = log.((1 .- outlier).*exp.(log_p) .+ outlier./outlier_vol)
log_p = sum(log_p_or_any ,dims=2)
log_p = dropdims(log_p, dims=(2,3,4))
size(log_p)



(2,)

In [34]:
dropdims(log_p, dims=(2,3,4))

2×3×3 CuArray{Float64, 3, CUDA.Mem.DeviceBuffer}:
[:, :, 1] =
 -0.692291  -0.6917    -0.691869
 -0.697296  -0.696199  -0.696595

[:, :, 2] =
 -8.14596  -8.12666  -8.13306
 -8.24099  -8.21395  -8.22389

[:, :, 3] =
 -3.78055  -3.77587  -3.77729
 -3.81413  -3.80621  -3.80909